# Benchmark Vision Models on CIFake Dataset

This notebook benchmarks convolutional and transformer-based networks defined in `vision_models.py` on the [CIFake dataset](https://www.kaggle.com/datasets/birdy654/cifake-real-and-ai-generated-synthetic-images).


In [ ]:
!pip install -q kaggle torchvision

In [ ]:
import os
from pathlib import Path
import numpy as np
import torch
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score
from pipelines_torch.benchmark import BenchmarkRunner
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from pipelines_torch.vision_models import MODEL_REGISTRY
from utils.metrics import METRIC_REGISTRY

DATA_DIR = Path('cifake_data')

In [ ]:
# Requires Kaggle API credentials available as environment variables
# KAGGLE_USERNAME and KAGGLE_KEY. See https://www.kaggle.com/docs/api.
if not DATA_DIR.exists():
    DATA_DIR.mkdir(parents=True, exist_ok=True)
    !kaggle datasets download -d birdy654/cifake-real-and-ai-generated-synthetic-images -p $DATA_DIR --unzip

In [ ]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
])

train_ds = datasets.ImageFolder(DATA_DIR / 'train', transform=transform)
X = torch.stack([img for img, _ in train_ds]).numpy()
y = np.array(train_ds.targets)
val_ds = datasets.ImageFolder(DATA_DIR / 'test', transform=transform)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=64)

class_names = train_ds.classes
num_classes = len(class_names)

In [ ]:
# Build model configs for BenchmarkRunner
model_configs = []
for name, model_class in MODEL_REGISTRY.items():
    # Skip models that might require special handling
    if name in ['qwen2_vl_qlora']:  # Skip complex models for now
        continue
    model_configs.append({
        "name": name,
        "class": model_class,
        "params": {"num_classes": num_classes}
    })

# Set up metrics
metrics = [METRIC_REGISTRY["accuracy"], METRIC_REGISTRY["f1"]]

# Configure BenchmarkRunner
runner = BenchmarkRunner(
    model_configs=model_configs,
    augmentations=[None],  # No augmentations for now
    metrics=metrics,
    task_type="classification",
    device="cuda" if torch.cuda.is_available() else "cpu",
    epochs=5,  # Reduced for faster testing
    batch_size=64,
    use_kfold=False,  # Single train/val split for speed
    learning_rate=1e-3,
    random_state=42
)

# Run benchmark
print(f"Running benchmark on {len(X)} samples with {num_classes} classes")
print(f"Data shape: {X.shape}")
print(f"Using device: {runner.device}")

results_df = runner.run(X, y)
print("\nBenchmark Results:")
print(results_df)